# XAI-GAN for Dorsal Hand Vein Authentication - Quick Start

This notebook provides a quick introduction to the XAI-GAN implementation.

## 1. Setup

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. Load Models

In [ ]:
from models.generator import Generator
from models.discriminator import Discriminator
from models.classifier import VeinAuthenticationClassifier
from utils.config import Config

config = Config()

# Initialize models
generator = Generator(
    latent_dim=config.LATENT_DIM,
    img_channels=config.IMG_CHANNELS,
    features=config.GEN_FEATURES
).to(device)

discriminator = Discriminator(
    img_channels=config.IMG_CHANNELS,
    features=config.DISC_FEATURES
).to(device)

classifier = VeinAuthenticationClassifier(
    num_classes=100  # Adjust based on your dataset
).to(device)

print(f'Generator parameters: {sum(p.numel() for p in generator.parameters()):,}')
print(f'Discriminator parameters: {sum(p.numel() for p in discriminator.parameters()):,}')
print(f'Classifier parameters: {sum(p.numel() for p in classifier.parameters()):,}')

## 3. Generate Synthetic Vein Patterns

In [ ]:
# Generate random vein patterns
generator.eval()
with torch.no_grad():
    noise = torch.randn(16, config.LATENT_DIM, device=device)
    fake_veins = generator(noise)

# Visualize
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    img = fake_veins[i, 0].cpu().numpy()
    img = (img + 1) / 2  # Denormalize from [-1, 1] to [0, 1]
    ax.imshow(img, cmap='gray')
    ax.axis('off')

plt.suptitle('Generated Vein Patterns', fontsize=16)
plt.tight_layout()
plt.show()

## 4. Test Grad-CAM Visualization

In [ ]:
from explainability.gradcam import GradCAM

# Create Grad-CAM explainer
gradcam = GradCAM(classifier, target_layer='layer4')

# Generate sample image
test_image = torch.randn(1, 1, 224, 224, device=device)

# Generate CAM
cam, output, pred_class = gradcam.generate_cam(test_image)

# Visualize
gradcam.visualize(test_image, cam)
plt.show()

print(f'Predicted class: {pred_class}')
print(f'CAM range: [{cam.min():.3f}, {cam.max():.3f}]')

## 5. Test SHAP Analysis

In [ ]:
from explainability.shap_analysis import VeinSHAPAnalyzer

# Create background data (small subset for demo)
background = torch.randn(20, 1, 224, 224, device=device)

# Create analyzer
shap_analyzer = VeinSHAPAnalyzer(classifier, background, device=device)
shap_analyzer.create_explainer(method='gradient')

# Explain instance
test_image = torch.randn(1, 1, 224, 224, device=device)
shap_values, pred = shap_analyzer.explain_instance(test_image)

# Visualize
shap_analyzer.visualize_shap_instance(test_image, shap_values)
plt.show()

print(f'Predicted class: {pred}')
print(f'SHAP values range: [{shap_values.min():.3f}, {shap_values.max():.3f}]')

## 6. Test LIME Explainability

In [ ]:
from explainability.lime_analysis import VeinLIMEAnalyzer

# Create analyzer
lime_analyzer = VeinLIMEAnalyzer(classifier, device=device)

# Generate explanation (reduced samples for speed)
test_image = torch.randn(1, 1, 224, 224, device=device)
explanation, pred, probs = lime_analyzer.explain_instance(
    test_image,
    num_samples=500  # Use 1000+ for production
)

# Visualize
lime_analyzer.visualize_explanation(test_image, explanation, pred)
plt.show()

print(f'Predicted class: {pred}')
print(f'Top 3 probabilities: {sorted(probs, reverse=True)[:3]}')

## 7. Test Fairness Analysis

In [ ]:
from fairness.bias_detection import FairnessAnalyzer
from torch.utils.data import TensorDataset, DataLoader

# Create analyzer
fairness_analyzer = FairnessAnalyzer(classifier, device=device)

# Create dummy demographic groups
def create_group_loader(num_samples=100):
    data = torch.randn(num_samples, 1, 224, 224)
    labels = torch.randint(0, 100, (num_samples,))
    dataset = TensorDataset(data, labels)
    return DataLoader(dataset, batch_size=16)

# Age groups
age_groups = {
    '18-30': create_group_loader(100),
    '31-50': create_group_loader(120),
    '51-75': create_group_loader(80)
}

# Analyze
age_analysis = fairness_analyzer.analyze_age_fairness(age_groups)

# Visualize
fairness_analyzer.visualize_fairness(age_analysis)
plt.show()

# Print results
print(f"Accuracy disparity: {age_analysis['parity']['accuracy_disparity']:.4f}")
print(f"Target threshold: 0.02 (2%)")
print(f"Compliant: {age_analysis['parity']['accuracy_disparity'] <= 0.02}")

## Next Steps

1. **Prepare Your Data:**
   - Download Jilin University and 11K Hands datasets
   - Run preprocessing scripts
   - Create train/val/test splits

2. **Train Models:**
   - Train GAN: `python training/train_gan.py`
   - Train Classifier: `python training/train_classifier.py`

3. **Run Full Analysis:**
   - Generate explainability visualizations
   - Conduct fairness analysis
   - Prepare user study

4. **Evaluate:**
   - Compare with baselines
   - Measure all metrics
   - Write results

See `IMPLEMENTATION_GUIDE.md` for detailed instructions!